In [11]:
import pandas as pd
from tqdm import tqdm
from utils import word_to_simple_identifier

In [12]:
df_sents = pd.read_parquet("../data/sentences.parquet")

In [13]:
big_word_dict = {}
for i, row in tqdm(df_sents.iterrows()):
    words = row["sentence_obj"]
    for word in words:
        key = word_to_simple_identifier(word)
        if key not in big_word_dict:
            big_word_dict[key] = {
                "text": word["string"],
                "count": 1,
                "lemma": word["lemma"],
                "pos": word["pos"],
                "features": word["features"],
                # "category": word["category"]
            }
        else:
            big_word_dict[key]["count"] += 1

127947it [00:06, 19249.88it/s]


In [14]:
word_to_lemma_df = pd.DataFrame.from_dict(big_word_dict, orient="index").sort_values(by="count", ascending=False).reset_index()#.drop([",", ".", ";", "·"])
word_to_lemma_df = pd.concat([word_to_lemma_df.drop(['features'], axis=1), word_to_lemma_df['features'].apply(pd.Series)], axis=1)
word_to_lemma_df = word_to_lemma_df[word_to_lemma_df["count"] > 1]
# word_to_lemma_df = pd.concat([word_to_lemma_df.drop(['category'], axis=1), word_to_lemma_df['category'].apply(pd.Series)], axis=1)

In [15]:
word_to_lemma_df = word_to_lemma_df[word_to_lemma_df.text.str.isalpha()]

In [16]:
word_to_lemma_df = word_to_lemma_df[word_to_lemma_df["text"] != "ΣΩ"] # remove misclassified from socrates

In [17]:
word_to_lemma_df.to_parquet("../data/word_to_lemma.parquet")